In [1]:
pip install roboflow

   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   -- ------------------------------------- 2.4/38.8 MB 13.4 MB/s eta 0:00:03
   ---- ----------------------------------- 4.7/38.8 MB 11.9 MB/s eta 0:00:03
   ------ --------------------------------- 6.0/38.8 MB 9.5 MB/s eta 0:00:04
   ------- -------------------------------- 6.8/38.8 MB 7.9 MB/s eta 0:00:05
   ------- -------------------------------- 7.3/38.8 MB 7.2 MB/s eta 0:00:05
   -------- ------------------------------- 7.9/38.8 MB 6.2 MB/s eta 0:00:05
   -------- ------------------------------- 8.4/38.8 MB 5.7 MB/s eta 0:00:06
   --------- ------------------------------ 9.2/38.8 MB 5.3 MB/s eta 0:00:06
   ---------- ----------------------------- 10.2/38.8 MB 5.3 MB/s eta 0:00:06
   ----------- ---------------------------- 11.5/38.8 MB 5.4 MB/s eta 0:00:06
   ------------- -------------------------- 13.1/38.8 MB 5.6 MB/s eta 0:00:05
   -------------- ------------------------- 14.4/38.8 MB 5.6 MB/s eta 0:00:05


In [ ]:
import roboflow

rf = roboflow.Roboflow(api_key="TxkrRT1VppZfE2dmANds")
project = rf.workspace("ved-2li6j").project("isl_dataset")
dataset = project.version("10").download("folder") 

loading Roboflow workspace...
loading Roboflow project...


RuntimeError: Version number 11 is not found.

In [6]:
import roboflow

rf = roboflow.Roboflow(api_key="TxkrRT1VppZfE2dmANds")
project = rf.workspace("ved-2li6j").project("isl_dataset")

# List all available versions
versions = project.versions()
print("Available Versions:")
for v in versions:
    print(f"Version: {v.version}")


loading Roboflow workspace...
loading Roboflow project...
Available Versions:
Version: 10
Version: 9
Version: 8
Version: 7
Version: 6
Version: 5
Version: 4
Version: 3
Version: 2
Version: 1


In [ ]:
# dataloader 
dataset = r"E:\images for phrases"


In [3]:
import os
import shutil
import random

# Original unsplit dataset path
original_dataset = r"E:\images for phrases"
split_dataset = r"E:\New folder\images for phrases_split"  # Output after split

# Split ratios
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# Create split folders
for split in ["train", "valid", "test"]:
    os.makedirs(os.path.join(split_dataset, split), exist_ok=True)

# Loop through each class
for class_name in os.listdir(original_dataset):
    class_dir = os.path.join(original_dataset, class_name)
    if not os.path.isdir(class_dir):
        continue

    images = [f for f in os.listdir(class_dir) if f.lower().endswith((".jpg", ".png", ".jpeg"))]
    random.shuffle(images)

    # Calculate split indices
    train_end = int(train_ratio * len(images))
    val_end = train_end + int(val_ratio * len(images))

    train_files = images[:train_end]
    val_files = images[train_end:val_end]
    test_files = images[val_end:]

    # Copy files into split folders
    for split_name, split_files in [("train", train_files), ("valid", val_files), ("test", test_files)]:
        split_class_dir = os.path.join(split_dataset, split_name, class_name)
        os.makedirs(split_class_dir, exist_ok=True)

        for file in split_files:
            shutil.copy(os.path.join(class_dir, file), os.path.join(split_class_dir, file))

print("✅ Dataset splitting complete!")


✅ Dataset splitting complete!


In [6]:
import os
import cv2
import mediapipe as mp
from tqdm import tqdm

# Paths
source_dataset = r"E:\New folder\images for phrases_split"  # Original dataset folder
target_dataset = r"E:\images_cropped"  # Cropped dataset output folder
target_size = (224, 224)  # Resize cropped hand(s) to this size

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands_detector = mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.3)
mp_drawing = mp.solutions.drawing_utils

# Create target directory if not exists
if not os.path.exists(target_dataset):
    os.makedirs(target_dataset)

def process_image(image_path, save_path):
    img = cv2.imread(image_path)
    if img is None:
        print(f"❌ Failed to load image: {image_path}")
        return False

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands_detector.process(img_rgb)

    if results.multi_hand_landmarks:
        h, w, _ = img.shape
        x_min, y_min, x_max, y_max = w, h, 0, 0

        # Combine bounding boxes of all detected hands
        for hand_landmarks in results.multi_hand_landmarks:
            for lm in hand_landmarks.landmark:
                x, y = int(lm.x * w), int(lm.y * h)
                x_min, y_min = min(x, x_min), min(y, y_min)
                x_max, y_max = max(x, x_max), max(y, y_max)

        # Add padding around combined box
        pad = 30
        x_min, y_min = max(0, x_min - pad), max(0, y_min - pad)
        x_max, y_max = min(w, x_max + pad), min(h, y_max + pad)

        # Crop combined hand region and resize
        hand_crop = img[y_min:y_max, x_min:x_max]
        if hand_crop.size == 0:
            print(f"⚠️ Empty crop (skipping): {image_path}")
            return False

        hand_crop = cv2.resize(hand_crop, target_size)

        # Save cropped image
        cv2.imwrite(save_path, hand_crop)
        return True
    else:
        print(f"⚠️ No hands detected: {image_path}")
        return False

# Walk through dataset
for split in ["train", "valid", "test"]:
    split_dir = os.path.join(source_dataset, split)
    target_split_dir = os.path.join(target_dataset, split)

    for class_name in os.listdir(split_dir):
        class_dir = os.path.join(split_dir, class_name)
        target_class_dir = os.path.join(target_split_dir, class_name)
        os.makedirs(target_class_dir, exist_ok=True)

        image_files = [f for f in os.listdir(class_dir) if f.lower().endswith((".jpg", ".png", ".jpeg"))]
        print(f"\n📦 Processing {split}/{class_name} ({len(image_files)} images)")

        for img_name in tqdm(image_files, desc=f"{split}/{class_name}"):
            src_img_path = os.path.join(class_dir, img_name)
            tgt_img_path = os.path.join(target_class_dir, img_name)

            success = process_image(src_img_path, tgt_img_path)

print("\n✅ Preprocessing complete. Cropped dataset saved to:")
print(target_dataset)



📦 Processing train/again (28 images)


train/again:   0%|          | 0/28 [00:00<?, ?it/s]

e:\ANACONDA3\envs\cv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
train/again: 100%|██████████| 28/28 [00:01<00:00, 23.01it/s]



📦 Processing train/agree (28 images)


train/agree: 100%|██████████| 28/28 [00:01<00:00, 21.01it/s]


...


📦 Processing test/write (6 images)


test/write:  50%|█████     | 3/6 [00:00<00:00, 21.59it/s]

⚠️ No hands detected: E:\New folder\images for phrases_split\test\write\19.png
⚠️ No hands detected: E:\New folder\images for phrases_split\test\write\31.png
⚠️ No hands detected: E:\New folder\images for phrases_split\test\write\32.png
⚠️ No hands detected: E:\New folder\images for phrases_split\test\write\5.png


test/write: 100%|██████████| 6/6 [00:00<00:00, 21.34it/s]


✅ Preprocessing complete. Cropped dataset saved to:
E:\images_cropped


In [ ]:
import os

source_dataset = r"E:\New folder\images for phrases_split"  
target_dataset = r"E:\images_cropped" 

for split in ["train", "valid", "test"]:
    print(f"\n📂 {split.upper()} SET")
    print(f"{'Class':<25} {'Original':<10} {'Cropped':<10}")

    split_original_dir = os.path.join(source_dataset, split)
    split_cropped_dir = os.path.join(target_dataset, split)

    for class_name in sorted(os.listdir(split_original_dir)):
        orig_dir = os.path.join(split_original_dir, class_name)
        crop_dir = os.path.join(split_cropped_dir, class_name)

        orig_count = len([f for f in os.listdir(orig_dir) if f.lower().endswith((".jpg", ".png", ".jpeg"))])
        crop_count = len([f for f in os.listdir(crop_dir) if f.lower().endswith((".jpg", ".png", ".jpeg"))]) if os.path.exists(crop_dir) else 0

        print(f"{class_name:<25} {orig_count:<10} {crop_count:<10}")



📂 TRAIN SET
Class                     Original   Cropped   
again                     28         28        
agree                     28         28        
answer                    28         28        
attendance                28         28        
book                      28         25        
break                     28         28        
careful                   28         16        
change                    28         27        
chat                      28         18        
congratulations           28         13        
email                     28         28        
file                      28         28        
good morning              28         27        
happy birthday            28         28        
home                      28         27        
how are you               28         27        
hungry                    28         0         
i need help               28         26        
join                      28         28        
keepsmile                 2

In [ ]:
import os
import shutil

source_dataset = r"E:\New folder\images for phrases_split"  
target_dataset = r"E:\images_cropped"  # Cropped dataset

classes_to_remove = set()

# Step 1: Find classes with 0 in any split
for split in ["train", "valid", "test"]:
    split_cropped_dir = os.path.join(target_dataset, split)
    if not os.path.exists(split_cropped_dir):
        continue

    for class_name in os.listdir(split_cropped_dir):
        class_dir = os.path.join(split_cropped_dir, class_name)
        img_count = len([f for f in os.listdir(class_dir) if f.lower().endswith((".jpg", ".png", ".jpeg"))])
        if img_count == 0:
            classes_to_remove.add(class_name)

# Step 2: Remove those classes from all splits in cropped dataset
for split in ["train", "valid", "test"]:
    for class_name in classes_to_remove:
        dir_to_remove = os.path.join(target_dataset, split, class_name)
        if os.path.exists(dir_to_remove):
            shutil.rmtree(dir_to_remove)
            print(f"🗑 Removed empty class: {split}/{class_name}")

print("\n✅ Cleanup complete!")


🗑 Removed empty class: train/valid
🗑 Removed empty class: train/train
🗑 Removed empty class: train/test
🗑 Removed empty class: train/hungry
🗑 Removed empty class: valid/valid
🗑 Removed empty class: valid/train
🗑 Removed empty class: valid/test
🗑 Removed empty class: valid/hungry
🗑 Removed empty class: test/valid
🗑 Removed empty class: test/train
🗑 Removed empty class: test/test
🗑 Removed empty class: test/hungry

✅ Cleanup complete!
